In [1]:
#!pip install nasdaq-data-link
import nasdaqdatalink
nasdaqdatalink.ApiConfig.api_key = '5BwoEH6x8VTcx9HS8hca'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn import tree
from sklearn.metrics import ConfusionMatrixDisplay
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
AAPLtest = nasdaqdatalink.get_table('SHARADAR/SEP', ticker=['AAPL'])
AAPLtrain = pd.read_csv('WIKIAAPL.csv')

In [3]:
AAPLtest

,ticker,date,open,high,low,close,volume,closeadj,closeunadj,lastupdated
None,,,,,,,,,,
0,AAPL,2018-12-31,39.633,39.840,39.120,39.435,140013864.0,37.801,157.74,2024-02-09
1,AAPL,2018-12-28,39.375,39.630,38.638,39.057,169165696.0,37.439,156.23,2024-02-09
2,AAPL,2018-12-27,38.960,39.193,37.517,39.038,212468260.0,37.420,156.15,2024-02-09
3,AAPL,2018-12-26,37.075,39.307,36.680,39.292,234330176.0,37.664,157.17,2024-02-09
4,AAPL,2018-12-24,37.038,37.888,36.648,36.708,148676928.0,35.186,146.83,2024-02-09
...,...,...,...,...,...,...,...,...,...,...
77,AAPL,2018-09-10,55.237,55.462,54.117,54.583,158065812.0,52.138,218.33,2024-02-09
78,AAPL,2018-09-07,55.462,56.343,55.178,55.325,150479240.0,52.847,221.30,2024-02-09
79,AAPL,2018-09-06,56.557,56.837,55.325,55.775,137159904.0,53.277,223.10,2024-02-09


In [4]:
def up_or_down(row):
    if row['Open'] < row['Close']:
        val = 1
    else:
        val = 0
    return val
def percentgain(row):
    val = ((row['open']-row['close'])/row['close'])*100
    return val
AAPLtrain['increased'] = AAPLtrain.apply(up_or_down, axis = 1)
#AAPL['percentgain'] = AAPL.apply(percentgain,axis =1)

In [5]:
def drop_columns(data, columns_to_drop):
    """
    drops the given columns from the dataset
    returns the new data without the column(s)
    """
    cleaned_data = data.drop(columns=columns_to_drop, axis=1)
    return cleaned_data

AAPLtrain = drop_columns(AAPLtrain, ['Ex-Dividend', 'Split Ratio', 'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'])
AAPLtest = drop_columns(AAPLtest, ['closeadj', 'closeunadj', 'lastupdated', 'ticker'])

In [6]:
def process_date_columns(data, date_column_name, reference_date_str='%d-%m-%Y'):
    """
    converts a date column into 4 separate columns: the day, month, and year with a fourth column countin the days since a reference date
    returns the dataframe but with the additional new 4 columns
    """
    data[date_column_name] = pd.to_datetime(data[date_column_name])

    data['year'] = data[date_column_name].dt.year
    data['month'] = data[date_column_name].dt.month
    data['day'] = data[date_column_name].dt.day

    reference_date = pd.to_datetime(reference_date_str, format='%d-%m-%Y')
    data['days_difference'] = (data[date_column_name] - reference_date).dt.days

    return data

AAPLtest = process_date_columns(AAPLtest, 'date', reference_date_str='04-09-2018')

In [7]:
AAPLtest

,date,open,high,low,close,volume,year,month,day,days_difference
None,,,,,,,,,,
0,2018-12-31,39.633,39.840,39.120,39.435,140013864.0,2018,12,31,118
1,2018-12-28,39.375,39.630,38.638,39.057,169165696.0,2018,12,28,115
2,2018-12-27,38.960,39.193,37.517,39.038,212468260.0,2018,12,27,114
3,2018-12-26,37.075,39.307,36.680,39.292,234330176.0,2018,12,26,113
4,2018-12-24,37.038,37.888,36.648,36.708,148676928.0,2018,12,24,111
...,...,...,...,...,...,...,...,...,...,...
77,2018-09-10,55.237,55.462,54.117,54.583,158065812.0,2018,9,10,6
78,2018-09-07,55.462,56.343,55.178,55.325,150479240.0,2018,9,7,3
79,2018-09-06,56.557,56.837,55.325,55.775,137159904.0,2018,9,6,2


In [8]:
AAPLtrain

,Date,Open,High,Low,Close,Volume,increased
0,2018-03-27,173.68,175.15,166.92,168.340,38962839.0,0
1,2018-03-26,168.07,173.10,166.44,172.770,36272617.0,1
2,2018-03-23,168.39,169.92,164.94,164.940,40248954.0,0
3,2018-03-22,170.00,172.68,168.60,168.845,41051076.0,0
4,2018-03-21,175.04,175.09,171.26,171.270,35247358.0,0
...,...,...,...,...,...,...,...
9395,1980-12-18,26.63,26.75,26.63,26.630,327900.0,0
9396,1980-12-17,25.87,26.00,25.87,25.870,385900.0,0
9397,1980-12-16,25.37,25.37,25.25,25.250,472000.0,0
9398,1980-12-15,27.38,27.38,27.25,27.250,785200.0,0


In [9]:
AAPLtrain = process_date_columns(AAPLtrain, 'Date', reference_date_str='12-12-1980')

In [10]:
AAPLtrain

,Date,Open,High,Low,Close,Volume,increased,year,month,day,days_difference
0,2018-03-27,173.68,175.15,166.92,168.340,38962839.0,0,2018,3,27,13619
1,2018-03-26,168.07,173.10,166.44,172.770,36272617.0,1,2018,3,26,13618
2,2018-03-23,168.39,169.92,164.94,164.940,40248954.0,0,2018,3,23,13615
3,2018-03-22,170.00,172.68,168.60,168.845,41051076.0,0,2018,3,22,13614
4,2018-03-21,175.04,175.09,171.26,171.270,35247358.0,0,2018,3,21,13613
...,...,...,...,...,...,...,...,...,...,...,...
9395,1980-12-18,26.63,26.75,26.63,26.630,327900.0,0,1980,12,18,6
9396,1980-12-17,25.87,26.00,25.87,25.870,385900.0,0,1980,12,17,5
9397,1980-12-16,25.37,25.37,25.25,25.250,472000.0,0,1980,12,16,4
9398,1980-12-15,27.38,27.38,27.25,27.250,785200.0,0,1980,12,15,3
